In [366]:
import spacy
from spacy.tokens import Doc, Token, Span
import pandas as pd
import numpy as np
from spacy import displacy
from spacy.matcher import Matcher
from spacy.pipeline import EntityRuler
from spacy_lookup import Entity
from pysbd.utils import PySBDFactory
# %run combining_models.ipynb
# %run doc_functions.ipynb

In [367]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { height:100% !important; }</style>"))

In [368]:
%run doc_functions.ipynb

In [369]:
#nlp1 = spacy.load("eng_bc5mdr")

In [370]:
nlp1 = spacy.load('en_ner_bc5cdr_md')

In [371]:
nlp1.pipe_names

['tagger', 'parser', 'ner']

In [372]:
matcher = Matcher(nlp1.vocab)

In [373]:
data = pd.read_csv('inclusion_criteria_2.csv',sep='|')

In [374]:
#data = pd.read_csv('data/sample_inclusion.csv',sep='|')

In [375]:
data.shape

(10740, 3)

In [376]:
data['inclusion'] = [text.replace('Inclusion Criteria:','') for text in data['inclusion']]
data['inclusion'] = [text.replace('Inclusion criteria:','') for text in data['inclusion']]
data['inclusion'] = [text.replace('inclusion criteria:','') for text in data['inclusion']]
data['inclusion'] = symbol_cleanup(data['inclusion'])

In [377]:
#setting token extensions
Token.set_extension('has_therapy',default=False,force=True)
Token.set_extension('has_treatment', default=False,force=True)
#Setting Doc extensions
Doc.set_extension('has_therapy',  default=False,force=True)
Doc.set_extension('has_treatment', default=False,force=True)
Doc.set_extension('has_diagnosis', default='',force=True)
Doc.set_extension('therapyList', default='',force=True) # this will be a list
Doc.set_extension('diseaseList', default='',force=True) # this will be a list
Doc.set_extension('chemicalList', default='',force=True) # this will be a list

In [378]:
# lookup = pd.read_csv(r'data\lookup1.csv',encoding='latin1')
lookup = pd.read_csv(r'data\lookup_clean.csv')
# lookup = lookup[lookup['DataDictionaryName'] != 'AECAT']
lookup = lookup.drop_duplicates(subset='UserDataString', keep='first')
# lookup = lookup[lookup.Include == 'Yes']
lookups = [pd.DataFrame(y) for x, y in lookup.groupby('DataDictionaryName', as_index=False)]


In [379]:
lookup.shape

(5216, 5)

In [380]:
lookup_grp = lookup.groupby('UserDataString').count()

In [381]:
entities = []
lookupUnits = []
lookupTexts = []
for lookup in lookups:
    lookupUnit = list(set(lookup.DataDictionaryName))[0]
    lookupText = list(lookup.UserDataString)
#     print(lookupUnit)
#     print(lookupText)
    entity = Entity( keywords_list =lookupText,label = lookupUnit, case_sensitive = False, )
    lookupUnits.append(lookupUnit)
    lookupTexts.append(lookupText)
    entities.append(entity)
#     print(lookupUnit)

In [382]:
#lookupText

for index,entity in enumerate(entities):
    entity_name = 'entity' + str(index)
    nlp1.remove_pipe(entity_name)

In [383]:
for index,entity in enumerate(entities):
    entity_name = 'entity' + str(index)
    nlp1.add_pipe(entity, name=entity_name,before='ner')

In [384]:
nlp1.add_pipe(PySBDFactory(nlp1), first=True)
nlp1.add_pipe(doc_attributes,first=True)
nlp1.add_pipe(token_attributes,first=True)

nlp1.add_pipe(getTherapyList,last=True)
nlp1.add_pipe(getDiseaseList,last=True)

In [385]:
print(nlp1.pipe_names)

['token_attributes', 'doc_attributes', 'PySBDFactory', 'tagger', 'parser', 'entity0', 'entity1', 'entity2', 'entity3', 'entity4', 'entity5', 'entity6', 'entity7', 'entity8', 'entity9', 'entity10', 'entity11', 'entity12', 'entity13', 'entity14', 'entity15', 'entity16', 'entity17', 'entity18', 'entity19', 'entity20', 'entity21', 'entity22', 'entity23', 'entity24', 'entity25', 'entity26', 'entity27', 'entity28', 'entity29', 'entity30', 'entity31', 'entity32', 'entity33', 'entity34', 'entity35', 'entity36', 'entity37', 'entity38', 'entity39', 'entity40', 'ner', 'getTherapyList', 'getDiseaseList']


In [386]:
removedLookup = pd.read_csv(r'data\removed_lookup.csv')
lookupPatterns = getPatterns(removedLookup)

In [387]:
ruler = EntityRuler(nlp1,overwrite_ents=True)
ruler.add_patterns(genderPatterns)
ruler.add_patterns(agePatterns)
ruler.add_patterns(comparePatterns)
ruler.add_patterns(rangePatterns)
ruler.add_patterns(numberPattern)
ruler.add_patterns(ecogPatterns)
ruler.add_patterns(lifePatterns)
ruler.add_patterns(measurePatterns)
ruler.add_patterns(diagnosisPatterns)
ruler.add_patterns(cancerPatterns)
ruler.add_patterns(subjectPatterns)
ruler.add_patterns(unitPatterns)
ruler.add_patterns(methodPatterns)
ruler.add_patterns(therapyPatterns)
ruler.add_patterns(treatmentPatterns)
ruler.add_patterns(occurancePattern)
ruler.add_patterns(severityPatterns)
ruler.add_patterns(eligibilityPatterns)
ruler.add_patterns(lookupPatterns)
nlp1.add_pipe(ruler,name='ruler1',before='ner')
# newruler = EntityRuler(nlp1,overwrite_ents=True)
# newruler.add_patterns(diagTermPatterns)
# nlp1.add_pipe(newruler,name='ruler2',after='ner')

In [388]:
# Topic >= 0 gets everything. Change to get only specific topic
sampledata = data[:500]

In [389]:
docs = list(nlp1.pipe(sampledata.inclusion))

In [390]:
sampledata = sampledata.assign(hasDiagnosis=   [doc._.has_diagnosis for doc in docs],
                  hasTherapy=     [doc._.has_therapy for doc in docs],
                  hasTreatment =  [doc._.has_treatment for doc in docs],
                  TherapyList =   [doc._.therapyList for doc in docs],
                  disease =       [doc._.diseaseList for doc in docs],
                  chemical =      [doc._.chemicalList for doc in docs],
                  sentenceCount = [len(list(doc.sents)) for doc in docs]
                 )

In [391]:
sampledata.to_csv('therapyAdded.csv',sep = '|')

In [392]:
sampledata.shape

(500, 10)

In [399]:
for index, doc in enumerate(docs):

    for sent in doc.sents:
        if any([ent.label_ == 'MEASURE' for ent in sent.ents]):
            print("\n")
            print()
            print('No:',index,':',doc)

            displacy.render(sent,style='ent')
#                     print("Sentence  :",sent)
#             print("NounChunks:",list(sent.noun_chunks))
            print("--------------------------------------------------------------------------------------")





No: 2 : 2. Patients with Eastern Cooperative Oncology Group (ECOG) performance status ≤ 1.


--------------------------------------------------------------------------------------



No: 8 : 8. Adequate renal function: serum creatinine ≤ 1.5 x upper limit of normal (ULN) (OR creatinine clearance [Cockcroft and Gault] ≥ 30 mL/min for participant with creatinine levels > 1.5 × ULN) within 14 days prior inclusion.


--------------------------------------------------------------------------------------



No: 9 : 9. Adequate liver function: aspartate aminotransferase (AST) and alanine aminotransferase (ALT) levels ≤ 3 × ULN ( ≤ 5 ULN when documented liver metastases) and total bilirubin level ≤ 1.5 × ULN, within 14 days prior inclusion.


--------------------------------------------------------------------------------------



No: 10 : 10. Adequate bone marrow function: absolute neutrophil count (ANC) ≥ 1,000/mm³, platelet count ≥ 100,000/mm³, and hemoglobin ≥ 9 g/dL, within 14 days prior inclusion.


--------------------------------------------------------------------------------------



No: 30 : 6. ECOG performance status of 0, 1, or


...

--------------------------------------------------------------------------------------



No: 483 : - Patients with adequate bone marrow function measured within 28 days prior to administration of study treatment as defined below: - Absolute neutrophil count ≥ 1.0 x 109 /L - Platelet count ≥ 50 x 109 /L - Haemoglobin ≥ 8.0 g/dL (may have been blood transfused) - Patients with adequate renal function: - Calculated Glomerular Filtration Rate (GFR) ≥ 30 ml/min/1.73 m2 according to CKD-EPI formula - Patients with adequate hepatic function - Total bilirubin ≤ 1 × upper limit of normal [ULN] and aspartate aminotransferase (AST) > ULN - Total bilirubin > 1.0 to1.5 × ULN and any AST - Patients must have a life expectancy ≥ 16 week.


--------------------------------------------------------------------------------------



No: 490 : - STEP 0 PRE-REGISTRATION INCLUSION CRITERIA - Patient must have active advanced melanoma, defined as unresectable stage IIIB-IV by American Joint Committee on Cancer (AJCC) 8th edition - Patient must have melanoma originating from cutaneous, acral-lentiginous, or mucosal primary sites.


--------------------------------------------------------------------------------------



No: 493 : Permitted systemic anti-PD-1 therapy regimens include: - Nivolumab 240 mg IV every (Q)2weeks or 480 mg IV Q4weeks - Pembrolizumab 200 mg IV Q3weeks or 400 mg IV Q6weeks - Nivolumab 1 mg/kg plus ipilimumab 3 mg/kg IV Q3weeks induction x 4 doses, followed by nivolumab 240 mg IV Q2weeks or 480 mg IV Q4weeks maintenance - Nivolumab 3 mg/kg plus Ipilimumab 1 mg/kg IV Q3weeks induction x 4 doses, followed by nivolumab 240 mg IV Q2weeks or 480 mg IV Q4weeks maintenance - Pembrolizumab 2 mg/kg (or 200 mg flat dose) plus Ipilimumab 1 mg/kg IV Q3weeks induction x 4 doses, followed by pembrolizumab 200 mg IV Q3weeks or 400 mg IV Q6weeks maintenance - Patient must have an Eastern Cooperative Oncology Group (ECOG) performance status of 0-2 - Human immunodeficiency virus (HIV)-infected patients on effective anti-retroviral therapy with undetectable viral load within 6 months are eligible for this tria

--------------------------------------------------------------------------------------



No: 498 : Anti-coagulation therapy is defined as low molecular weight heparin, warfarin, factor Xa inhibitor, or direct thrombin inhibitor - Have documentation of inability to perform the biopsy due to feasibility or safety concerns - Leukocytes > = 3,000/mcL (obtained = < 4 weeks prior to protocol registration) - Absolute neutrophil count > = 1,500/mcL (obtained = < 4 weeks prior to protocol registration) - Platelets > = 100,000/mcL (obtained = < 4 weeks prior to protocol registration) - Total bilirubin = < institutional upper limit of normal (ULN) (patients with history of Gilbert's syndrome are permitted to have a total bilirubin > 1.5 x institutional ULN) (obtained = < 4 weeks prior to protocol registration) - Aspartate aminotransferase (AST) (serum glutamic-oxaloacetic transaminase [SGOT])/alanine aminotransferase (ALT) (serum glutamate pyruvate transaminase [SGPT]) = < 2.5 x institutional UL

--------------------------------------------------------------------------------------


In [394]:
doc = nlp1("Histological diagnosis of extensive stage small cell lung cancer (ES-SCLC).")

In [395]:
docu = docs[63]
for doc in docu.sents:
    displacy.render(doc,style='ent',options={"color":"blue"})
#     print("Sentence  :",doc)
    print("NounChunks:",list(doc.noun_chunks))
for token in doc:
    print(token.text, token.pos_, token.is_oov, token.lemma_)


NounChunks: [8. Subjects, they]
8 NUM False 8
. PUNCT False .
Subjects NOUN False subject
with ADP False with
brain NOUN False brain
metastases NOUN False metastasis
are AUX False be
eligible ADJ False eligible
if SCONJ False if
they PRON False -PRON-
are AUX False be
asymptomatic ADJ False asymptomatic
, PUNCT False ,
are AUX False be
treated VERB False treat
or CCONJ False or
are AUX False be
neurological ADJ False neurological
stable ADJ False stable
for ADP False for
at ADP False at
least ADJ False least
2 NUM False 2
weeks NOUN False week
without ADP False without
the DET False the
use NOUN False use
of ADP False of
steroids NOUN False steroid
or CCONJ False or
on ADP False on
stable ADJ False stable
or CCONJ False or
decreasing VERB False decrease
dose NOUN False dose
of ADP False of
< X False <
10 NUM False 10
mg NOUN False mg
daily ADJ False daily
prednisone NOUN False prednisone
or CCONJ False or
equivalent ADJ False equivalent
. PUNCT False .


In [396]:
spacy.explain('DET')

'determiner'

In [397]:
doc = nlp1('no prior experience')

In [398]:
for token in doc:
    print(token.text, token.pos_, token.is_oov, token.lemma_)

no DET False no
prior ADJ False prior
experience NOUN False experience
